In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from pathlib import Path
sns.set_style("whitegrid")

In [3]:
#tweets_trump=pd.read_csv('./tweet_data/trump.csv',engine='python')
#tweets_biden=pd.read_csv('./tweet_data/biden.csv')
#tjb=tweets_biden.dropna(subset=['lat','long','city','state','user_location'],how='all')
#tdt=tweets_trump.dropna(subset=['lat','long','city','state','user_location'],how='all')
#tjb=tjb.loc[(tjb['country']=="United States" ) | (tjb['country']=="United States of America")]
#tdt=tdt.loc[(tdt['country']=="United States" ) | (tdt['country']=="United States of America")]
#photon_url="http://localhost:2322/api?q="
#photon_reverse="http://localhost:2322/reverse?"

#tjb['query_url']=pd.NA
#for x in tjb.index:
#    if pd.notna(tjb.loc[x,'lat']) and pd.notna(tjb.loc[x,'long']):
#        tjb.loc[x,'query_url']=photon_reverse+'lat='+str(tjb.loc[x,'lat'])+'&'+'lon='+str(tjb.loc[x,'long'])
#    elif pd.notna(tjb.loc[x,'city']) and pd.notna(tjb.loc[x,'state']):
#        tjb.loc[x,'query_url']=photon_url+ tjb.loc[x,'city']+ ' ' + tjb.loc[x,'state']
#    else:
#        tjb.loc[x,'query_url']=photon_url+str(tjb.loc[x,'user_location'])
#tjb['county']=pd.NA
#i=0
#j=0
#for x in tjb.index:
#    try:
#        photon_response=requests.get(tjb.loc[x,'query_url']).json()['features'][0]['properties']['county']
        #j=j+1
#    except:
#        photon_response=pd.NA
#    tjb.loc[x,'county'] = photon_response
    #i=i+1
    #print(i,j)
    #tdt['query_url']=pd.NA
#for x in tdt.index:
#    if pd.notna(tdt.loc[x,'lat']) and pd.notna(tdt.loc[x,'long']):
#        tdt.loc[x,'query_url']=photon_reverse+'lat='+str(tdt.loc[x,'lat'])+'&'+'lon='+str(tdt.loc[x,'long'])
#    elif pd.notna(tjb.loc[x,'city']) and pd.notna(tdt.loc[x,'state']):
#        tdt.loc[x,'query_url']=photon_url+ tdt.loc[x,'city']+ ' ' + tdt.loc[x,'state']
#    else:
#        tdt.loc[x,'query_url']=photon_url+str(tdt.loc[x,'user_location'])
#tdt['county']=pd.NA
#i=0
#j=0
#for x in tdt.index:
#    try:
#        photon_response=requests.get(tdt.loc[x,'query_url']).json()['features'][0]['properties']['county']
#        j=j+1
#    except:
#        photon_response=pd.NA
#    tdt.loc[x,'county'] = photon_response
#    i=i+1
#    print(i,j)
#filepath=Path('./tweet_data/biden_countied.csv')
#tjb.to_csv(filepath)
#filepath=Path('./tweet_data/trump_countied.csv')
#tdt.to_csv(filepath)

In [4]:
#Read in the files
#tweets_biden=pd.read_csv('./tweet_data/biden_countied.csv')
#tweets_trump=pd.read_csv('./tweet_data/trump_countied.csv')
#Only consider those with a valid county and cut down on not needed columns
#valid_tweets_b=tweets_biden.dropna(subset=['county']).drop(axis=1,labels=['lat','long','city','country','continent','state','collected_at','user_screen_name','user_name','source','tweet_id','query_url','user_join_date','Unnamed: 0'])
#valid_tweets_t=tweets_trump.dropna(subset=['county']).drop(axis=1,labels=['lat','long','city','country','continent','state','collected_at','user_screen_name','user_name','source','tweet_id','query_url','user_join_date','Unnamed: 0'])
#valid_tweets=pd.concat([valid_tweets_b,valid_tweets_t], ignore_index=True)
#valid_tweets.to_csv('./tweet_data/valid_tweets.csv')


Following is some boilerplate code to cleanup and name the dataset from census

In [12]:
demographics=pd.read_csv('./demographics/demographics.csv',engine='python')
tmp_in=pd.Series(range(1,39)).apply(str).apply(lambda x:x.zfill(3))
columns_to_drop1=[x+y+z for x in ('S0101_C'+pd.Series(['02','04','05','06'])+'_') for y in tmp_in for z in ['E','M']]
columns_to_drop2=[x+y+z for x in ('S0101_C'+pd.Series(['01','03'])+'_') for y in tmp_in for z in ['M']]
a=list(range(20,39))
a.remove(23)
ttmp_in=pd.Series(a).apply(str).apply(lambda x:x.zfill(3))
columns_to_drop3=[x+y+z for x in ('S0101_C'+pd.Series(['01','03'])+'_') for y in ttmp_in for z in ['E']]
columns_to_drop=columns_to_drop1+columns_to_drop2+columns_to_drop3+['Unnamed: 458']
#S0101_C01_023E
#().apply(lambda x:x+tmp_in+'E')
demographics=demographics.drop(axis=1,labels=columns_to_drop)
def convert_column_names(x):
    x_split = x.split('_')
    if x == 'S0101_C01_023E':
        return 'pop_18_24'
    if len(x_split)!=3:
        return x
    if x_split[1]=='C01':
        i=int(x_split[2][:-1])
        if i==1:
            return 'total_pop'
        return 'pop_'+str((i-2)*5)+'_'+str((i-1)*5)
    if x_split[1]=='C03':
        i=int(x_split[2][:-1])
        if i==1:
            return 'total_male_pop'
        return 'pop_male_'+str((i-2)*5)+'_'+str((i-1)*5)
    
demographics.rename(axis=1,mapper=convert_column_names,inplace=True)
demographics=demographics.loc[1:,]
demographics.to_csv('./demographics/demographics_cleaned.csv')



In [11]:
demographics.loc[0,'pop_18_24']

'Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!18 to 24 years'

In [16]:
dp=pd.read_csv('./demographics/dp_2016.csv',engine='python')
valid_columns=['GEO_ID','NAME','DP05_0033E','DP05_0034E','DP05_0039E','DP05_0066E','DP05_0032E','DP05_0028E']
dp_vc=dp.loc[:,valid_columns]
dp_vc.rename(axis=1,mapper={'DP05_0033E':'afr_amer','DP05_0034E':'amer_ind','DP05_0039E':'asian','DP05_0066E':'latino','DP05_0032E':'white','DP05_0028E':'tot_pop'},inplace=True)
dp_vc=dp_vc.loc[1:]
dp_vc.replace(to_replace='N',value=0,inplace=True)
for x in dp_vc.columns:
    if x!='NAME' and x!='GEO_ID':
        dp_vc[x]=dp_vc[x].apply(int)
dp_vc.to_csv('./demographics/dp_2016_cleaned.csv')

In [15]:
dp_vc.head(5)

,GEO_ID,NAME,afr_amer,amer_ind,asian,latino,white,tot_pop
0,Geography,Geographic Area Name,Estimate!!RACE!!One race!!Black or African Ame...,Estimate!!RACE!!One race!!American Indian and ...,Estimate!!RACE!!One race!!Asian,Estimate!!HISPANIC OR LATINO AND RACE!!Total p...,Estimate!!RACE!!One race!!White,Estimate!!RACE!!Total population
1,0500000US01001,"Autauga County, Alabama",10196,226,444,1416,42311,55049
2,0500000US01003,"Baldwin County, Alabama",18594,1355,1338,8712,172441,199510
3,0500000US01005,"Barbour County, Alabama",12756,46,129,1147,12430,26614
4,0500000US01007,"Bibb County, Alabama",4788,81,14,502,17370,22572


In [20]:
dp=pd.read_csv('./demographics/dp_2019.csv',engine='python')
valid_columns=['GEO_ID','NAME','DP05_0038E','DP05_0039E','DP05_0044E','DP05_0071E','DP05_0037E','DP05_0033E']
dp_vc=dp.loc[:,valid_columns]
dp_vc.rename(axis=1,mapper={'DP05_0038E':'afr_amer','DP05_0039E':'amer_ind','DP05_0044E':'asian','DP05_0071E':'latino','DP05_0037E':'white','DP05_0033E':'tot_pop'},inplace=True)
dp_vc=dp_vc.loc[1:]
dp_vc.replace(to_replace='N',value=0,inplace=True)
for x in dp_vc.columns:
    if x!='NAME' and x!='GEO_ID':
        dp_vc[x]=dp_vc[x].apply(int)
dp_vc.to_csv('./demographics/dp_2019_cleaned.csv')

In [21]:
dp_vc.head(5)

,GEO_ID,NAME,afr_amer,amer_ind,asian,latino,white,tot_pop
1,0500000US01001,"Autauga County, Alabama",10538,140,573,1565,42527,55380
2,0500000US01003,"Baldwin County, Alabama",19718,1645,1969,9711,183471,212830
3,0500000US01005,"Barbour County, Alabama",12066,82,134,1105,11869,25361
4,0500000US01007,"Bibb County, Alabama",5014,30,27,579,17272,22493
5,0500000US01009,"Blount County, Alabama",928,46,212,5342,55062,57681


In [37]:
#Cleanup Poverty Data

poverty=pd.read_csv('./poverty/poverty_2016.csv',engine='python')
valid_columns=['GEO_ID','NAME','S1701_C02_001E','S1701_C01_040E']
poverty_vc=poverty.loc[:,valid_columns]
poverty_vc.rename(axis=1,mapper={'S1701_C02_001E':'tot_poverty','S1701_C01_040E':'pov_150'},inplace=True)
poverty_vc=poverty_vc.loc[1:]
for x in poverty_vc.columns:
    if x!='NAME' and x!='GEO_ID':
        poverty_vc[x]=poverty_vc[x].apply(int)
    #poverty_vc['tot_poverty'].apply(int)
#poverty_vc['pov_18_up']=poverty_vc['pov_18_64']+poverty_vc['pov_65_up']
poverty_vc.to_csv('./poverty/poverty_2016_cleaned.csv')

In [45]:
income=pd.read_csv('./income/income_2016.csv',engine='python')
valid_columns=['GEO_ID','NAME','DP03_0009PE','DP03_0062E','DP03_0063E']
income_vc=income.loc[:,valid_columns]
income_vc.rename(axis=1,mapper={'DP03_0009PE':'unemp_rate','DP03_0062E':'med_hh_inc','DP03_0063E':'mean_hh_inc'},inplace=True)
income_vc=income_vc.loc[1:]
income_vc.replace(to_replace='N',value=0,inplace=True)
for x in income_vc.columns:
    if x!='NAME' and x!='GEO_ID':
        income_vc[x]=income_vc[x].apply(float)
income_vc.to_csv('./income/income_2016_cleaned.csv')

In [46]:
income_vc[:5]

,GEO_ID,NAME,unemp_rate,med_hh_inc,mean_hh_inc
1,0500000US01001,"Autauga County, Alabama",5.6,53099.0,67651.0
2,0500000US01003,"Baldwin County, Alabama",6.3,51365.0,70277.0
3,0500000US01005,"Barbour County, Alabama",12.8,33956.0,45280.0
4,0500000US01007,"Bibb County, Alabama",7.1,39776.0,53863.0
5,0500000US01009,"Blount County, Alabama",6.0,46212.0,56439.0
